In [5]:
import pandas as pd
from datetime import datetime
from typing import Optional
import re

In [8]:
import pandas as pd
from datetime import datetime
from typing import Optional
import re

class TravelLinkGenerator:
    def __init__(self, stations_file='all_stations.csv', airports_file='airports_data.csv'):
        """Initialize with the paths to the stations and airports CSV files."""
        try:
            # Read train stations CSV
            self.stations_df = pd.read_csv(stations_file)
            self.stations_df['Station Name'] = self.stations_df['Station Name'].astype(str).str.lower()
            self.stations_df['Station Code'] = self.stations_df['Station Code'].astype(str).str.upper()
            
            # Read airports CSV
            self.airports_df = pd.read_csv(airports_file)
            self.airports_df['City'] = self.airports_df['City'].astype(str).str.lower()
            self.airports_df['IATA Code'] = self.airports_df['IATA Code'].astype(str).str.upper()
            
        except FileNotFoundError as e:
            raise FileNotFoundError(f"Required file not found: {e.filename}")

    def get_station_code(self, city_name: str) -> str:
        """Get train station code from city name."""
        city_name = city_name.lower().strip()
        
        # First try exact match
        station = self.stations_df[self.stations_df['Station Name'] == city_name]
        
        # If no exact match, try contains with word boundaries
        if station.empty:
            pattern = f'\b{re.escape(city_name)}\b'
            station = self.stations_df[self.stations_df['Station Name'].str.contains(pattern, case=False, regex=True, na=False)]
        
        if station.empty:
            raise ValueError(
                f"\nTrain station code not found for city: {city_name}\n"
                f"Please try searching with a different city name."
            )
        
        # Sort by number of trains to get the main station
        station = station.sort_values('Trains passing through', ascending=False)
        return station.iloc[0]['Station Code']

    def get_airport_code(self, city_name: str) -> str:
        """Get airport code from city name."""
        city_name = city_name.lower().strip()
        
        # First try exact match
        airport = self.airports_df[self.airports_df['City'] == city_name]
        
        # If no exact match, try contains with word boundaries
        if airport.empty:
            pattern = f'\b{re.escape(city_name)}\b'
            airport = self.airports_df[self.airports_df['City'].str.contains(pattern, case=False, regex=True, na=False)]
        
        if airport.empty:
            raise ValueError(
                f"\nAirport code not found for city: {city_name}\n"
                f"Please try searching with a different city name."
            )
        
        # Filter for valid IATA codes and sort by importance
        airport = airport[airport['IATA Code'].notna()]
        airport = airport[airport['IATA Code'] != 'nan']
        if airport.empty:
            raise ValueError(f"No airport with IATA code found for city: {city_name}")
        
        # Sort by priority (international airports first)
        if 'Type' in airport.columns:
            airport = airport.sort_values('Type', ascending=False)
        
        return airport.iloc[0]['IATA Code']

    def _format_date(self, date_str: str) -> str:
        """Convert date from YYYY-MM-DD to DDMMYYYY format."""
        try:
            date_obj = datetime.strptime(date_str, "%Y-%m-%d")
            if date_obj < datetime.now().replace(hour=0, minute=0, second=0, microsecond=0):
                raise ValueError("Date cannot be in the past")
            return date_obj.strftime("%d%m%Y")
        except ValueError as e:
            if "time data" in str(e):
                raise ValueError("Invalid date format. Please use YYYY-MM-DD format.")
            raise e

    def generate_flight_link(
        self,
        from_city: str,
        to_city: str,
        from_date: str,
        to_date: Optional[str] = None,
        adults: int = 1,
        children: int = 0,
        infants: int = 0,
        class_type: str = "e"
    ) -> str:
        """Generate flight search link using city names."""
        # Get airport codes
        from_code = self.get_airport_code(from_city)
        to_code = self.get_airport_code(to_city)
        
        if from_code == to_code:
            raise ValueError("Departure and arrival cities cannot be the same")

        # Validate passenger counts
        if adults < 1:
            raise ValueError("At least one adult passenger is required")
        if children < 0 or infants < 0:
            raise ValueError("Passenger counts cannot be negative")
        if adults + children + infants > 9:
            raise ValueError("Total number of passengers cannot exceed 9")
        if infants > adults:
            raise ValueError("Number of infants cannot exceed number of adults")

        # Format dates
        formatted_from_date = self._format_date(from_date)
        if to_date:
            formatted_to_date = self._format_date(to_date)
            date_param = f"{formatted_from_date}/{formatted_to_date}"
        else:
            date_param = formatted_from_date
        
        # Generate link
        base_url = "https://www.ixigo.com/search/result/flight"
        link = (
            f"{base_url}?"
            f"from={from_code}&"
            f"to={to_code}&"
            f"date={date_param}&"
            f"adults={adults}&"
            f"children={children}&"
            f"infants={infants}&"
            f"class={class_type}&"
            f"source=Search+Form&"
        )
        
        return link

    def generate_train_link(
        self,
        from_city: str,
        to_city: str,
        from_date: str,
        to_date: Optional[str] = None,
        adults: int = 1,
        children: int = 0,
        seniors: int = 0,
        students: int = 0,
        class_type: str = "ALL"
    ) -> str:
        """Generate train search link using city names."""
        # Get station codes
        from_code = self.get_station_code(from_city)
        to_code = self.get_station_code(to_city)
        
        if from_code == to_code:
            raise ValueError("Departure and arrival cities cannot be the same")

        # Validate passenger counts
        if adults < 1:
            raise ValueError("At least one adult passenger is required")
        if any(count < 0 for count in [children, seniors, students]):
            raise ValueError("Passenger counts cannot be negative")
        if adults + children + seniors + students > 6:
            raise ValueError("Total number of passengers cannot exceed 6")

        # Format dates
        formatted_from_date = self._format_date(from_date)
        if to_date:
            formatted_to_date = self._format_date(to_date)
            date_param = f"{formatted_from_date}/{formatted_to_date}"
        else:
            date_param = formatted_from_date
        
        # Generate link
        base_url = "https://www.ixigo.com/search/result/train"
        link = (
            f"{base_url}/{from_code}/{to_code}/{date_param}//"
            f"{adults}/{children}/{seniors}/{students}/{class_type}"
        )
        
        return link

def get_user_input():
    """Get travel details from user input."""
    print("\n=== Travel Link Generator ===")
    print("Enter travel details (press Enter to use defaults):")
    
    from_city = "Delhi"
    to_city = "Mumbai"
    from_date = "2025-04-30"
    to_date = "2025-05-04"
    
    # Get passenger counts
    adults = 2
    children = 3 
    infants = 0
    seniors = 0
    students = 0
    
    return {
        "from_city": from_city,
        "to_city": to_city,
        "from_date": from_date,
        "to_date": to_date,
        "adults": adults,
        "children": children,
        "infants": infants,
        "seniors": seniors,
        "students": students
    }

# Example usage
if __name__ == "__main__":
    try:
        # Create generator instance
        generator = TravelLinkGenerator('all_stations.csv', 'airports_data.csv')
        
        # Get user input
        travel_details = get_user_input()
        
        # Generate flight link
        flight_link = generator.generate_flight_link(
            from_city=travel_details["from_city"],
            to_city=travel_details["to_city"],
            from_date=travel_details["from_date"],
            to_date=travel_details["to_date"],
            adults=travel_details["adults"],
            children=travel_details["children"],
            infants=travel_details["infants"]
        )
        print("\nGenerated Flight Search Link:")
        print(flight_link)
        
        # Generate train link
        train_link = generator.generate_train_link(
            from_city=travel_details["from_city"],
            to_city=travel_details["to_city"],
            from_date=travel_details["from_date"],
            to_date=travel_details["to_date"],
            adults=travel_details["adults"],
            children=travel_details["children"],
            seniors=travel_details["seniors"],
            students=travel_details["students"]
        )
        print("\nGenerated Train Search Link:")
        print(train_link)
        
    except ValueError as e:
        print(f"\nError: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}") 


=== Travel Link Generator ===
Enter travel details (press Enter to use defaults):

Generated Flight Search Link:
https://www.ixigo.com/search/result/flight?from=NAN&to=BOM&date=30042025/04052025&adults=2&children=3&infants=0&class=e&source=Search+Form&

Error: 
Train station code not found for city: delhi
Please try searching with a different city name.
